In [3]:
!aws s3 ls s3://amazon-reviews-pds/tsv/

2017-11-24 13:22:50          0 
2017-11-24 13:48:03  241896005 amazon_reviews_multilingual_DE_v1_00.tsv.gz
2017-11-24 13:48:17   70583516 amazon_reviews_multilingual_FR_v1_00.tsv.gz
2017-11-24 13:48:34   94688992 amazon_reviews_multilingual_JP_v1_00.tsv.gz
2017-11-24 13:49:14  349370868 amazon_reviews_multilingual_UK_v1_00.tsv.gz
2017-11-24 13:48:47 1466965039 amazon_reviews_multilingual_US_v1_00.tsv.gz
2017-11-24 13:49:53  648641286 amazon_reviews_us_Apparel_v1_00.tsv.gz
2017-11-24 13:56:36  582145299 amazon_reviews_us_Automotive_v1_00.tsv.gz
2017-11-24 14:04:02  357392893 amazon_reviews_us_Baby_v1_00.tsv.gz
2017-11-24 14:08:11  914070021 amazon_reviews_us_Beauty_v1_00.tsv.gz
2017-11-24 14:17:41 2740337188 amazon_reviews_us_Books_v1_00.tsv.gz
2017-11-24 14:45:50 2692708591 amazon_reviews_us_Books_v1_01.tsv.gz
2017-11-24 15:10:21 1329539135 amazon_reviews_us_Books_v1_02.tsv.gz
2017-11-24 15:22:13  442653086 amazon_reviews_us_Camera_v1_00.tsv.gz
2017-11-24 15:27:13 2689739299 amazon_rev

In [4]:
!aws s3 ls s3://amazon-reviews-pds/parquet/

                           PRE product_category=Apparel/
                           PRE product_category=Automotive/
                           PRE product_category=Baby/
                           PRE product_category=Beauty/
                           PRE product_category=Books/
                           PRE product_category=Camera/
                           PRE product_category=Digital_Ebook_Purchase/
                           PRE product_category=Digital_Music_Purchase/
                           PRE product_category=Digital_Software/
                           PRE product_category=Digital_Video_Download/
                           PRE product_category=Digital_Video_Games/
                           PRE product_category=Electronics/
                           PRE product_category=Furniture/
                           PRE product_category=Gift_Card/
                           PRE product_category=Grocery/
                           PRE product_category=Health_&_Personal_Care/
   

In [33]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [23]:
!aws s3 mb s3://amazon-reviews-capstone

make_bucket: amazon-reviews-capstone


In [25]:
s3_public_source_tsv = "s3://amazon-reviews-pds/tsv"

In [26]:
%store s3_source_tsv

Stored 's3_source_tsv' (str)


In [27]:
s3_private_source_tsv = "s3://amazon-reviews-capstone/amazon-reviews-pds/tsv".format(bucket)
print(s3_private_source_tsv)

s3://amazon-reviews-capstone/amazon-reviews-pds/tsv


In [28]:
%store s3_private_source_tsv

Stored 's3_private_source_tsv' (str)


In [29]:
!aws s3 cp --recursive $s3_public_source_tsv/ $s3_private_source_tsv/

copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_UK_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_UK_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/ts

In [38]:
import sys
!{sys.executable} -m pip install PyAthena
db_name = "amazon_reviews_capstone"


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [40]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir = "s3://{0}/athena/staging".format(bucket),
               region_name="us-east-1")



In [41]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(db_name)
print(statement)

CREATE DATABASE IF NOT EXISTS amazon_reviews_capstone


In [42]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [49]:
table_name_tsv = "amazon_reviews_tsv"

# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_tsv(
    marketplace string,
    customer_id string,
    review_id string,
    product_id string,
    product_parent string,
    product_title string,
    product_category string,
    star_rating int,
    helpful_votes int,
    total_votes int,
    vine string,
    verified_purchase string,
    review_headline string,
    review_body string,
    review_date string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
    LINES TERMINATED BY '\n'
    LOCATION 's3://amazon-reviews-capstone/amazon-reviews-pds/tsv'
    TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')"""

pd.read_sql(statement, conn)
print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_tsv(
    marketplace string,
    customer_id string,
    review_id string,
    product_id string,
    product_parent string,
    product_title string,
    product_category string,
    star_rating int,
    helpful_votes int,
    total_votes int,
    vine string,
    verified_purchase string,
    review_headline string,
    review_body string,
    review_date string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY '	'
    LINES TERMINATED BY '
'
    LOCATION 's3://amazon-reviews-capstone/amazon-reviews-pds/tsv'
    TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [51]:
df = pd.read_sql("SELECT * FROM amazon_reviews_capstone.amazon_reviews_tsv limit 8;", conn)
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,10140119,R3LI5TRP3YIDQL,B00TXH4OLC,384427924,Whatever's for Us: Remastered,Music,5,0,0,N,Y,Five Stars,Love this CD along with other CDs by the same ...,2015-08-31
1,US,27664622,R3LGC3EKEG84PX,B00B6QXN6U,831769051,Same Trailer Different Park,Music,5,0,0,N,Y,A new fave in our house,This is the album that introduced me to Kacey ...,2015-08-31
2,US,45946560,R9PYL3OYH55QY,B001GCZXW6,14067376,Soaring (Jazz Club),Music,5,0,1,N,Y,Five Stars,Excellent / thanks,2015-08-31
3,US,15146326,R3PWBAWUS4NT0Q,B000003EK6,566295619,CARIBBEAN PARTY,Music,3,0,0,N,Y,Three Stars,Nice variety of caribbean artists.,2015-08-31
4,US,16794688,R15LYP3O51UU9E,B00N1F0BKK,210426072,Pain Killer,Music,5,0,0,N,Y,Five Stars,Purchased as a gift and they loved this cd,2015-08-31
5,US,32203364,R1AD7L0CC3DSRI,B00V7KAO7Q,360249504,A Thoughtiverse Unmarred,Music,5,0,0,N,Y,Definitely worth a listen.,Really enjoyed the content of this album. I b...,2015-08-31
6,US,1194276,R32FE8Y45QV434,B000094Q4P,20972283,Comfort of Her Wings,Music,5,0,0,N,Y,Five Stars,Great CD,2015-08-31
7,US,45813052,R3NM4MZ4XWL43Q,B00JMK0P1I,378049560,Sleep Soundly CD - Calming Guitar Music with N...,Music,5,1,2,N,Y,Five Stars,Excellent CD!,2015-08-31


In [ ]:
table_name_parquet = "amazon_reviews_parquet"
# Set S3 path to Parquet data
s3_path_parquet = "s3://amazon-reviews-capstone/amazon-reviews-pds/parquet".format(bucket)
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://amazon-reviews-capstone/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM amazon_reviews_capstone.amazon_reviews_tsv"""
pd.read_sql(statement, conn)
print(statement)